In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

--2024-02-13 19:53:02--  https://cdn.freecodecamp.org/project-data/books/book-crossings.zip
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 104.26.2.33, 172.67.70.149, 104.26.3.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|104.26.2.33|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 26085508 (25M) [application/zip]
Saving to: ‘book-crossings.zip.1’

book-crossings.zip. 100%[===================>]  24.88M  54.9MB/s    in 0.5s    

2024-02-13 19:53:03 (54.9 MB/s) - ‘book-crossings.zip.1’ saved [26085508/26085508]

Archive:  book-crossings.zip
replace BX-Book-Ratings.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: All
  inflating: BX-Book-Ratings.csv     
  inflating: BX-Books.csv            
  inflating: BX-Users.csv            


In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
#information about df_books
print(df_books.head())
print(df_books.shape)
print(df_books.isna().sum())



         isbn                                              title  \
0  0195153448                                Classical Mythology   
1  0002005018                                       Clara Callan   
2  0060973129                               Decision in Normandy   
3  0374157065  Flu: The Story of the Great Influenza Pandemic...   
4  0393045218                             The Mummies of Urumchi   

                 author  
0    Mark P. O. Morford  
1  Richard Bruce Wright  
2          Carlo D'Este  
3      Gina Bari Kolata  
4       E. J. W. Barber  
(271379, 3)
isbn      0
title     0
author    1
dtype: int64


In [ ]:
print(df_ratings.head())
print(df_ratings.shape)
print(df_ratings['rating'].unique())

print(df_ratings.isna().sum())

     user        isbn  rating
0  276725  034545104X     0.0
1  276726  0155061224     5.0
2  276727  0446520802     0.0
3  276729  052165615X     3.0
4  276729  0521795028     6.0
(1149780, 3)
[ 0.  5.  3.  6.  8.  7. 10.  9.  4.  1.  2.]
user      0
isbn      0
rating    0
dtype: int64


In [ ]:
#since there is only one book with a missing author , delete that row
df_books.dropna(inplace = True)

print(df_books.isna().sum())

df = pd.merge(df_books , df_ratings , how = 'left' , left_on= 'isbn' , right_on = 'isbn')

print(df)
print(df.isna().sum())

isbn      0
title     0
author    0
dtype: int64
               isbn                                              title  \
0        0195153448                                Classical Mythology   
1        0002005018                                       Clara Callan   
2        0002005018                                       Clara Callan   
3        0002005018                                       Clara Callan   
4        0002005018                                       Clara Callan   
...             ...                                                ...   
1032378  0440400988                         There's a Bat in Bunk Five   
1032379  0525447644                            From One to One Hundred   
1032380  006008667X  Lily Dale : The True Story of the Town that Ta...   
1032381  0192126040                        Republic (World's Classics)   
1032382  0767409752  A Guided Tour of Rene Descartes' Meditations o...   

                       author      user  rating  
0          M

In [ ]:
#removing users with <200 ratings and books with <100 ratings
df = df_ratings

counts1 = df['user'].value_counts()
counts2 = df['isbn'].value_counts()

df = df[~df['user'].isin(counts1[counts1 < 200].index)]
df = df[~df['isbn'].isin(counts2[counts2 < 100].index)]

df = pd.merge(right=df, left = df_books, on="isbn")

df = df.drop_duplicates(["title", "user"])

In [ ]:
#building pivot table
pivot = df.pivot_table(index = "title", columns = "user", values = "rating")
pivot.fillna(0, inplace=True)

In [ ]:
#generating model
model = NearestNeighbors( algorithm = 'brute' ,metric = 'cosine').fit(pivot.values)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book = ""):
  x=pivot.loc[book].array.reshape(1, -1)
  distances,indices=model.kneighbors(x,n_neighbors=6)
  R_books=[]
  for distance,indice in zip(distances[0],indices[0]):
    if distance!=0:
      R_book=pivot.index[indice]
      R_books.append([R_book,distance])
  recommended_books=[book,R_books[::-1]]

  return recommended_books

In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()

["Where the Heart Is (Oprah's Book Club (Paperback))", [["I'll Be Seeing You", 0.8016211], ['The Weight of Water', 0.77085835], ['The Surgeon', 0.7699411], ['I Know This Much Is True', 0.7677075], ['The Lovely Bones: A Novel', 0.7234864]]]
You passed the challenge! 🎉🎉🎉🎉🎉
